In [2]:
!pip install kagglehub --quiet

import kagglehub
import pandas as pd
import os


In [3]:
# Dataset: Superstore Sales Dataset (Rohit Sahoo)
# Enlace: https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting

# Descargamos el dataset y obtenemos la ruta local
dataset_path = kagglehub.dataset_download("rohitsahoo/sales-forecasting")
print("✅ Dataset descargado en:", dataset_path)

100%|██████████| 480k/480k [00:00<00:00, 62.6MB/s]

Extracting files...
✅ Dataset descargado en: /root/.cache/kagglehub/datasets/rohitsahoo/sales-forecasting/versions/2


In [4]:
# Buscamos el archivo principal en la carpeta descargada
csv_files = [f for f in os.listdir(dataset_path) if f.endswith(".csv")]
if len(csv_files) == 0:
    raise FileNotFoundError("❌ No se encontró ningún archivo CSV en el dataset.")
else:
    print("Archivos CSV encontrados:", csv_files)

# Seleccionamos el primer archivo CSV (por lo general es 'train.csv')
file_path = os.path.join(dataset_path, csv_files[0])
print("Archivo a cargar:", file_path)

Archivos CSV encontrados: ['train.csv']
Archivo a cargar: /root/.cache/kagglehub/datasets/rohitsahoo/sales-forecasting/versions/2/train.csv


In [5]:
# Cargamos los datos con Pandas
df = pd.read_csv(file_path)

In [6]:
# EXPLORACIÓN INICIAL
# ==============================================
print("\nPrimeras 5 filas del dataset:")
print(df.head())

print("\nInformación general del dataset:")
print(df.info())

print("\nNúmero de filas y columnas:", df.shape)



Primeras 5 filas del dataset:
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Regi

In [7]:
# LIMPIEZA Y PREPARACIÓN DE DATOS
# ==============================================

# --- Paso 1: Eliminación de duplicados
filas_antes = df.shape[0]
df = df.drop_duplicates()
filas_despues = df.shape[0]
print(f"\nRegistros duplicados eliminados: {filas_antes - filas_despues}")

# --- Paso 2: Manejo de valores faltantes
print("\nValores nulos por columna antes de limpieza:")
print(df.isnull().sum())

columnas_clave = [col for col in ['Order ID', 'Order Date', 'Ship Date'] if col in df.columns]
df = df.dropna(subset=columnas_clave)

df = df.fillna("No disponible")


Registros duplicados eliminados: 0

Valores nulos por columna antes de limpieza:
Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64


In [8]:
# --- Paso 3: Estandarización de categorías
for col in df.select_dtypes(include='object').columns:
    if col not in ['Order ID','Customer ID','Product ID']:
        df[col] = df[col].astype(str).str.title()

# --- Paso 4: Normalización de fechas (solo si existen)
if 'Order Date' in df.columns:
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
if 'Ship Date' in df.columns:
    df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce')
    df = df.dropna(subset=['Order Date','Ship Date'])

In [9]:
# --- Paso 5: Creación de columnas derivadas (si Order Date existe)
if 'Order Date' in df.columns:
    df['Order Year'] = df['Order Date'].dt.year
    df['Order Month'] = df['Order Date'].dt.month
    df['Order Quarter'] = df['Order Date'].dt.to_period("Q")
if 'Order Date' in df.columns and 'Ship Date' in df.columns:
    df['Ship Lead Time (Days)'] = (df['Ship Date'] - df['Order Date']).dt.days

In [10]:
# RESULTADO FINAL
# ==============================================
print("\nInformación después de limpieza:")
print(df.info())

print("\nPrimeras 5 filas después de limpieza y columnas derivadas:")
print(df.head())

print("\nNúmero final de filas y columnas:", df.shape)


Información después de limpieza:
<class 'pandas.core.frame.DataFrame'>
Index: 2676 entries, 0 to 9786
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Row ID                 2676 non-null   int64         
 1   Order ID               2676 non-null   object        
 2   Order Date             2676 non-null   datetime64[ns]
 3   Ship Date              2676 non-null   datetime64[ns]
 4   Ship Mode              2676 non-null   object        
 5   Customer ID            2676 non-null   object        
 6   Customer Name          2676 non-null   object        
 7   Segment                2676 non-null   object        
 8   Country                2676 non-null   object        
 9   City                   2676 non-null   object        
 10  State                  2676 non-null   object        
 11  Postal Code            2676 non-null   object        
 12  Region                 2676 non-n